# Ephemerides

The ephemerides are the position end velocity Cartesian vectors of a moving object. In `pykep` the class {py:class}`~pykep.planet` offers a common interface to access various ephemerides, regardless on how they are computed. Whether the underlying computations are simply Keplerian, or based on more advanced trajectory propagations, interpolations or predictions, a unified interface is offered by this type-erasing class.

The user can code his own python class following the mandatory interface of a {py:class}`~pykep.planet` and his own coded objects will be treated uniformly as any other planet in `pykep`. These User Defined Planets (or UPLAs) can implement heterogeneous techniques and interface to any third-party API, once they are used to construct a {py:class}`~pykep.planet`, they will appear as any other moving object in `pykep`.

For convenience, a number of already coded UDPLAs are offered so that, to start with, users can compute the positions of planet, satellites comets and spacecarft without the need to code their own UDPLA.

In [2]:
import pykep as pk

Let us start to use the UDPLA {py:class}`~pykep.udpla.keplerian` which describes the motion of an object in a Keplerian orbit. All UDPLAS provided by `pykep` are in the module {py:mod}`~pykep.udpla`

In [3]:
udpla = pk.udpla.keplerian?

Docstring:  keplerian udpla
Type:       pybind11_type
Subclasses: 